- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

In [ ]:
import pandas as pd
import altair as alt
from datetime import datetime
import calendar

In [ ]:
df = pd.read_excel('2014-2020.xlsx')
df.head()

In [ ]:
df["hour"] = df.apply(lambda row: int(row["Час/Дата"].split('-')[0]), axis = 1)

In [ ]:
df["day"] = df.apply(lambda row: int(row["Час/Дата"].split('-')[1].split('.')[0]), axis = 1)

In [ ]:
df["month"] = df.apply(lambda row: int(row["Час/Дата"].split('-')[1].split('.')[1]), axis = 1)

In [ ]:
df["year"] = df.apply(lambda row: int(row["Час/Дата"].split('-')[1].split('.')[2]), axis = 1)

In [ ]:
df["day_of_year"] = df.apply(lambda row: str(row["month"]) + "." + str(row["day"]),axis=1)

In [ ]:
df["weekday"] = df.apply(lambda row: calendar.day_name[datetime(row["year"], row["month"], row["day"]).weekday()],axis=1)

In [ ]:
seasons_dict = {"winter": [12,1,2], "spring": [3,4,5], "summer": [6,7,8], "autumn": [9,10,11]}

reverse_season_dict = {}
for k, v in seasons_dict.items(): 
    for i in v:
        reverse_season_dict[i] = k

In [ ]:
reverse_season_dict

In [ ]:
df["season"] = df.apply(lambda row: reverse_season_dict[row["month"]], axis=1)

In [ ]:
aes_df = df[["AES", "hour", "day", "month", "year", "season", "day_of_year", "weekday"]]
aes_df.rename({"AES": "gen"},axis=1, inplace=True)
aes_df["type"] = "AES"

tec_df = df[["TEC", "hour", "day", "month", "year", "season", "day_of_year", "weekday"]]
tec_df.rename({"TEC": "gen"},axis=1, inplace=True)
tec_df["type"] = "TEC"

vde_df = df[["VDE", "hour", "day", "month", "year", "season", "day_of_year", "weekday"]]
vde_df.rename({"VDE": "gen"},axis=1, inplace=True)
vde_df["type"] = "VDE"

tes_df = df[["TES", "hour", "day", "month", "year", "season", "day_of_year", "weekday"]]
tes_df.rename({"TES": "gen"},axis=1, inplace=True)
tes_df["type"] = "TES"

ges_df = df[["GES", "hour", "day", "month", "year", "season", "day_of_year", "weekday"]]
ges_df.rename({"GES": "gen"},axis=1, inplace=True)
ges_df["type"] = "GES"

gaes_df = df[["GAES_GEN", "hour", "day", "month", "year", "season", "day_of_year", "weekday"]]
gaes_df.rename({"GAES_GEN": "gen"},axis=1, inplace=True)
gaes_df["type"] = "GAES"



gen_df = pd.concat([aes_df, tec_df, vde_df, tes_df, ges_df, gaes_df], axis=0)

In [ ]:
gen_df.head()

In [ ]:
alt.data_transformers.disable_max_rows()

**Як змінювалась структура генерації електроенергії за роками?**

Я вибрав репрезентацію, що дозволяє побачити і сумарну кількість генерації енергії в Україні у кожен окремий рік, і частку кожного окремого типу енергії від загальної кількості. Таким чином наша візуалізація виконує одразу дві функції - демонструє зміну в часі та частку в цілому. Також я відсортував сегменти графіку, тип енергії з найбільшою кількістю енергії виробленою за рік був на першому місці, а з найменшою - на останньому.

Альтернативні варіанти:

Використати `mark_line` замість `mark_area`. Але тоді важче було б оцінити частку кожного типу енергії від цілого.

In [ ]:
alt.Chart(gen_df).transform_fold(
    ["type"],
).mark_area().encode(
    x = alt.X('year:Q', axis = alt.Axis(tickCount = 5, format = '<4d')),
    y = alt.Y('gen:Q', aggregate = 'sum'),
    color = alt.Color('type:N'),
    order = alt.Order('gen:Q', aggregate='sum', sort = 'ascending')
).properties(width = 800, height = 500, background = '#F9F9F9', padding = 25)

**Як залежить споживання електроенергії від дня року та години доби?**

Я вибрав варіант візуалізації за допомогою прямокутників, кожен з яких репрезентує споживання електроенергії в конкретний день року в конкретну годину. Також я відсортував дні за постачанням енергії, так що дні з більшим загальним постачанням за день показуються вище. Таким чином нам легко порівнювати загальне споживання за різні дні року. Недоліком є те, що для того щоб порівняти 2 конкретні дні нам треба в найгіршому випадку перебрати 365 днів. Але ми точно знаємо: який з цих днів вище у спису, у того споживання більше. Кольорова теплова карта відображає різницю споживання за різні години дня. Іншим недоліком такої репрезентації є її велика довжина, цю проблему можна було б вирішити за допомогою агрегації, скажімо, по тижням, але нашею задачею було показати залежність саме по днях

In [ ]:
cons_df = df[["hour", "day", "month", "year", "season", "day_of_year", "CONSUMPTION", "weekday"]]

In [ ]:
cons_df.rename({"CONSUMPTION": "consumption"},axis=1,inplace=True)

In [ ]:
cons_df.head()

In [ ]:
alt.Chart(cons_df).transform_fold(
    ["consumption"],
).mark_rect().encode(
    x = alt.X('hour:N'),
    y = alt.Y('day_of_year:N', sort = alt.Sort(field = 'consumption', op = 'sum', order = 'descending')),
    color = alt.Color('consumption:Q', scale = alt.Scale(scheme = 'pinkyellowgreen'))
).properties(width = 800)

**Як змінюється генерація електроенергії з різних джерел впродовж доби?**

Я вибрав варіант візуалізації за допомогою прямокутників, кожен з яких репрезентує середню генерацію електроенергії певним джерелом в конкретну годину. 

Альтернативним варіантом було використання гістограм: кожен стовпчик розбитий на частини, щоб відобразити частку генерації в цілому за певну годину часу, але інтуїтиіно це не мало багато інформативного сенсу, тому я зупинився на цьому варіанті

In [ ]:
alt.Chart(gen_df).transform_fold(
    ["gen"],
).mark_rect().encode(
    x = alt.X('hour:N'),
    y = alt.Y('type:N', sort = alt.Sort(field = 'gen', op = 'mean', order = 'descending')),
    color = alt.Color('gen:Q', scale = alt.Scale(scheme = 'pinkyellowgreen'))
).properties(width = 800)

**Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?**

Я вибрав варіант візуалізації за допомогою прямокутників, кожен з яких репрезентує середнє споживання електроенергії в конкретну годину, в певний місяць. Я згрупував місяці по порам року, щоб було зручніше порівнювати споживання в різні сезони. Очевидно, що взимку зеленого кольору найбільше, отже споживання в цей час найбільше.

Альтернативним варіантом було використання гісторгам, але велика їх кількість виглядала громіздко

In [ ]:
alt.Chart(cons_df).transform_fold(
    ["consumption"],
).mark_rect().encode(
    x = alt.X('hour:N'),
    y = alt.Y('month:O'),
    row = alt.Row('season:N'),
    color = alt.Color('consumption:Q', scale = alt.Scale(scheme = 'pinkyellowgreen'))
).properties(width = 800)

**Як змінюється споживання електроенергії впродовж тижня?**

Я вибрав простий спосіб візуалізації середнього споживання енергії в датасеті - гістограми, розташовані в рядок. 
Перевагою є те, що по висоті стовпчика легко порівнювати дні тижня.

Альтернативним варіантом було використати відображення за допомогою одного кольорового стовпчика, де кожен колір відображав би день тижня.
У даному випадку важко було б порівнювати споживання за часткою кольору, бо значення є дуже близькими. Тому тут я перевагу надав висоті стовпчика

In [ ]:
sort= ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [ ]:
alt.Chart(cons_df).mark_bar().encode(
    x = alt.X('weekday:N', sort=alt.Sort(field = 'consumption', op = 'mean', order = 'ascending')),
    y = alt.Y('consumption:Q', aggregate = 'mean'),
    tooltip = alt.Tooltip('consumption', aggregate = 'mean')
).properties(width = 800, height = 500)